#### Install dependencies

In [1]:
%pip install -U openmim
!mim install mmcv-full

# install mmpose dependencies
%pip install -v -r requirements.txt

%pip install mmdet
%pip install mmpose
# install mmpose in develop mode
%pip install -v -e .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.5/237.5 kB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 16.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
/usr/local/lib/python3.9/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Looking in links: https://download.openmmlab.com/mmcv/dist/cu116/torch1.12.0/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 MB 7.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.2/190.2 kB 19.4 MB/s eta 0:00:00
Using pip 22.2.2 from /usr/local/lib/python3.9/dist-packages/pip (python 3.9)
Ignoring dataclasses: markers 'python_version == "3.6"' don't match your environment
  Cloning https://github.com/svenkreiss/poseval.git to /tmp/pip-install-it2_dim2/poseval_f8b7999

#### Import libraries

In [2]:
import mmcv
from mmcv import Config, DictAction
import mmpose
import torch, torchvision
import time
import os
import shutil
import os.path as osp
import numpy as np
import pickle

import time
import warnings
import copy

from collections import OrderedDict, defaultdict
import torch.distributed as dist
from mmcv.runner import get_dist_info, init_dist, set_random_seed
from mmcv.utils import get_git_hash
from mmpose import __version__
from mmpose.apis import init_random_seed, train_model
from mmpose.models import build_posenet
from mmpose.utils import collect_env, get_root_logger, setup_multi_processes

from mmcv.cnn import fuse_conv_bn
from mmcv.parallel import MMDataParallel, MMDistributedDataParallel
from mmcv.runner import get_dist_info, init_dist, load_checkpoint

from mmpose.apis import multi_gpu_test, single_gpu_test
from mmpose.datasets import build_dataloader, build_dataset

try:
    from mmcv.runner import wrap_fp16_model
except ImportError:
    warnings.warn('auto_fp16 from mmpose will be deprecated from v0.15.0'
                  'Please install mmcv>=1.1.4')
    from mmpose.core import wrap_fp16_model
    
from matplotlib import pprint
from mmpose.apis import (inference_top_down_pose_model, init_pose_model,
                         vis_pose_result, process_mmdet_results)
from mmdet.apis import inference_detector, init_detector
from xtcocotools.coco import COCO

local_runtime = False
import cv2
from matplotlib import pyplot as plt
from mmpose.core.bbox.transforms import bbox_xywh2xyxy
from mmpose.datasets import DatasetInfo
import matplotlib
from mmpose.apis import (collect_multi_frames, get_track_id,
                         inference_top_down_pose_model, init_pose_model,
                         process_mmdet_results, vis_pose_tracking_result)

from mmpose.core import Smoother
from mmpose.apis import (collect_multi_frames, extract_pose_sequence,
                         get_track_id, inference_pose_lifter_model,
                         inference_top_down_pose_model, init_pose_model,
                         process_mmdet_results, vis_3d_pose_result)

from mmpose.core import Smoother
from mmpose.datasets import DatasetInfo
import pandas as pd
from PIL import Image
import os
import pandas as pd
import random
import math
import json


### 2D

#### Extract all AcinoSet 2D Labelled data: COCO format

In [ ]:
# NB: unzip "labelled_data.zip" to "/notebooks/data/acino/labelled" first

acino='/notebooks/data/acino'

img_ids=[os.path.splitext(img)[0] for img in os.listdir(acino+'/../ap10k/data/')]
img_ids.remove('.ipynb_checkpoints')
img_ids=[int(i) for i in img_ids]
img_no=max(img_ids)+1

base=pd.read_csv('base.csv')

for dir in os.listdir(acino+'/labelled'): #JamesFlick1 JamesFlick2 etc.
    print(dir)
    if os.path.isdir(acino+'/labelled/'+dir):
        
        df=pd.read_csv('labelled/'+dir+'/'+"CollectedData_UCT.csv")

        df.iloc[0,1:]=df.iloc[0,1:].astype(str) +'_'+ df.iloc[1,1:].astype(str)
        df.columns=df.iloc[0] # Make keypoint labels column headers
        df=df.iloc[2:,:]
        
        #Extract COCO keypoints (AP10K):
        #df=df.reindex(columns=(['bodyparts','l_eye_x','l_eye_y','r_eye_x','r_eye_y','nose_x','nose_y','neck_base_x','neck_base_y','tail_base_x','tail_base_y','l_shoulder_x','l_shoulder_y','l_front_knee_x','l_front_knee_y','l_front_paw_x','l_front_paw_y','r_shoulder_x','r_shoulder_y','r_front_knee_x','r_front_knee_y','r_front_paw_x','r_front_paw_y','l_hip_x','l_hip_y','l_back_knee_x','l_back_knee_y','l_back_paw_x','l_back_paw_y','r_hip_x','r_hip_y','r_back_knee_x','r_back_knee_y','r_back_paw_x','r_back_paw_y','tail2_x','tail2_y']))
        
        #Extract COCO keypoints (All Acino Keypoints):
        df=df.reindex(columns=(['bodyparts','l_eye_x','l_eye_y','r_eye_x','r_eye_y','nose_x','nose_y','neck_base_x','neck_base_y','tail_base_x','tail_base_y','l_shoulder_x','l_shoulder_y','l_front_knee_x','l_front_knee_y','l_front_paw_x','l_front_paw_y','r_shoulder_x','r_shoulder_y','r_front_knee_x','r_front_knee_y','r_front_paw_x','r_front_paw_y','l_hip_x','l_hip_y','l_back_knee_x','l_back_knee_y','l_back_paw_x','l_back_paw_y','r_hip_x','r_hip_y','r_back_knee_x','r_back_knee_y','r_back_paw_x','r_back_paw_y','tail2_x','tail2_y','tail1_x','tail1_y','r_front_ankle_x','r_front_ankle_y','l_front_ankle_x','l_front_ankle_y','r_back_ankle_x','r_back_ankle_y','l_back_ankle_x','l_back_ankle_y','spine_x','spine_y']))
        
        for i in range(0,len(df)): # Iterate over rows
            orig=df.iloc[i,0].split('/')[2]
            df.iloc[i,0]='as_'+str(img_no)+'.jpg'
            
            im=Image.open('data/acino/labelled/'+dir+'/'+orig)
            img_name='as_'+str(img_no)
            im.save('data/acino/data/'+img_name+'.jpg')
            
            #os.remove('labelled1/'+dir+'/'+orig)
            
            
            img_no+=1
        base=pd.concat([base,df[0:]],ignore_index=True)
               
base.to_csv("data/acino/annotations.csv",index=False)

In [ ]:
df = pd.read_csv('data/acino/annotations.csv')

if os.path.exists("data/acino/annotations/acino_all.json"):
    os.remove("data/acino/annotations/acino_all.json")
    
shutil.copyfile('data/acino/annotations/anns_empty_acino.json','data/acino/annotations/acino_all.json')

with open('data/acino/annotations/acino_all.json','r+') as f: # Save all annotation instances in one .json file
    anns_data=json.load(f)
    
    for i in range(0,len(df)):# Iterate over each image

        #Add image instance
        img_name=df.iloc[i,0]
        img_id=int(img_name[3:].split('.')[0])
        img=cv2.imread('data/acino/data/'+img_name)
        height = img.shape[0]
        width = img.shape[1]
        img_inst=dict(width=width,height=height,file_name=img_name,background=1,id=img_id)
        
        #Add annotation instance
        ann_id=img_id
        kp=[]
        num_kp=0

        # Initial bbox:
        x_min=width-1
        x_max=1
        y_min=height-1
        y_max=1

        for j in range(1,len(df.columns),2): # -12 for AP10k format
            if pd.isnull(df.iloc[i,j]): #NaN values
                kp.append(0)
                kp.append(0)
                kp.append(0)
                
            else:
                x=int(round(pd.to_numeric(df.iloc[i,j])))
                y=int(round(pd.to_numeric(df.iloc[i,j+1])))
                
                #Add annotations
                kp.append(x)
                kp.append(y)
                kp.append(2)
                num_kp+=1

                #Bbox
                if x<x_min:
                    x_min=x
                if x>x_max:
                    x_max=x
                if y>y_max:
                    y_max=y
                if y<y_min:
                    y_min=y
                    
        # Pad bbox with extra pixels:
        x_min=x_min-15
        y_min=y_min-15
        
        # BBox w, h
        w=max(1,(x_max-x_min))+30
        h=max(1,(y_max-y_min))+30
        a=w*h # Bbox area
        
        #Padded bounding box:
        bbox=[x_min,y_min,w,h] # xywh format NB! y_min==top left corner
        
        # Uncomment for AP10k format:
        
#         if kp[-1]==2: # If tip of tail != NaN
#             num_kp-=1 
            
#         del kp[-3:] # Remove tail_x, tail_y, visible
        
        
        ann_inst=dict(image_id=img_id,iscrowd=0,category_id=1,num_keypoints=num_kp,keypoints=kp,bbox=bbox,id=ann_id,area=a)
        anns_data['images'].append(img_inst)
        anns_data['annotations'].append(ann_inst)
    f.seek(0)
    json.dump(anns_data,f,indent=4)
f.close()

In [ ]:
%pip install echo1-coco-split

!coco-split --annotations_file data/acino/annotations/acino_all.json --valid_ratio 0.1 --test_ratio 0.2 --train_name data/acino/annotations/acino_train.json --valid_name data/acino/annotations/acino_val.json --test_name data/acino/annotations/acino_test.json --has_annotations

#### Train, test and demo 2D pose estimator

##### Config file:

In [5]:
# Config file:
cfg = Config.fromfile('configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/acino/res152_acino_256x256.py')
# print(cfg.work_dir)
# train_from_config(cfg,True)
train_from_config(cfg,True)
# cfg.lr_config

2022-10-15 14:09:02,000 - mmpose - INFO - Distributed training: False
2022-10-15 14:09:02,003 - mmpose - INFO - Set random seed to 0, deterministic: False


loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
=> num_images: 5346
=> load 5289 samples
load checkpoint from torchvision path: torchvision://resnet152


Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2022-10-15 14:09:12,620 - mmpose - INFO - Start running, host: root@nttgqxc8s2, work_dir: /notebooks/work_dirs/res152_acino_256x256_pretr
2022-10-15 14:09:12,623 - mmpose - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook                           
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook                           
(LOW         ) IterTimerHook                      
 ---

...

DONE (t=0.20s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] =  0.731
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] =  0.980
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] =  0.869
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] =  0.593
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] =  0.745
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] =  0.770
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] =  0.988
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] =  0.893
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] =  0.667
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] =  0.784


2022-10-15 16:05:42,549 - mmpose - INFO - Now best checkpoint is saved as best_AP_epoch_65.pth.
2022-10-15 16:05:42,558 - mmpose - INFO - Best AP is 0.7307 at 65 epoch.
2022-10-15 16:05:42,561 - mmpose - INFO - Epoch(val) [65][24]	AP: 0.7307, AP .5: 0.9796, AP .75: 0.8685, AP (M): 0.5930, AP (L): 0.7454, AR: 0.7699, AR .5: 0.9881, AR .75: 0.8930, AR (M): 0.6670, AR (L): 0.7839
2022-10-15 16:05:49,433 - mmpose - INFO - Epoch [66][5/166]	lr: 5.000e-05, eta: 0:25:14, time: 1.372, data_time: 0.789, memory: 9943, heatmap_loss: 0.0004, acc_pose: 0.8916, loss: 0.0004
2022-10-15 16:05:52,306 - mmpose - INFO - Epoch [66][10/166]	lr: 5.000e-05, eta: 0:25:10, time: 0.574, data_time: 0.003, memory: 9943, heatmap_loss: 0.0003, acc_pose: 0.9310, loss: 0.0003
2022-10-15 16:05:55,178 - mmpose - INFO - Epoch [66][15/166]	lr: 5.000e-05, eta: 0:25:07, time: 0.574, data_time: 0.002, memory: 9943, heatmap_loss: 0.0003, acc_pose: 0.9116, loss: 0.0003
2022-10-15 16:05:58,051 - mmpose - INFO - Epoch [66][20/1

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 757/757, 67.1 task/s, elapsed: 11s, ETA:     0sLoading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *keypoints*
DONE (t=0.20s).
Accumulating evaluation results...


2022-10-15 16:31:51,647 - mmpose - INFO - Epoch(val) [80][24]	AP: 0.7364, AP .5: 0.9797, AP .75: 0.8709, AP (M): 0.6115, AP (L): 0.7528, AR: 0.7777, AR .5: 0.9894, AR .75: 0.8970, AR (M): 0.6802, AR (L): 0.7910


DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] =  0.736
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] =  0.980
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] =  0.871
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] =  0.612
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] =  0.753
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] =  0.778
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] =  0.989
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] =  0.897
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] =  0.680
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] =  0.791


In [ ]:
# Config file:
cfg = Config.fromfile('configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/acino/res152_acino_256x256.py')
# print(cfg.work_dir)
# train_from_config(cfg,True)
train_from_config(cfg,False)
# cfg.lr_config

/notebooks/mmpose/utils/setup_env.py:32: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
/notebooks/mmpose/utils/setup_env.py:42: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
2022-10-15 20:21:04,845 - mmpose - INFO - Distributed training: False
2022-10-15 20:21:04,849 - mmpose - INFO - Set random seed to 0, deterministic: False


loading annotations into memory...
Done (t=0.10s)
creating index...
index created!
=> num_images: 5346
=> load 5289 samples


2022-10-15 20:21:09,818 - mmpose - INFO - Start running, host: root@nql0gx2gd8, work_dir: /notebooks/work_dirs/res152_acino_256x256
2022-10-15 20:21:09,842 - mmpose - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook                           
(LOW         ) IterTimerHook                      
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) EvalHook                           
(LOW         ) IterTimerHook                      
 -------------------- 
after_train_iter:
(ABOVE_NORMAL) OptimizerHook                      
(NORMAL      ) CheckpointHook

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
=> num_images: 764
=> load 757 samples


2022-10-15 20:21:24,200 - mmpose - INFO - Epoch [1][5/166]	lr: 4.496e-06, eta: 10:34:42, time: 2.869, data_time: 1.030, memory: 9942, heatmap_loss: 0.0016, acc_pose: 0.0091, loss: 0.0016
2022-10-15 20:21:27,107 - mmpose - INFO - Epoch [1][10/166]	lr: 9.491e-06, eta: 6:21:33, time: 0.582, data_time: 0.003, memory: 9942, heatmap_loss: 0.0017, acc_pose: 0.0120, loss: 0.0017
2022-10-15 20:21:30,012 - mmpose - INFO - Epoch [1][15/166]	lr: 1.449e-05, eta: 4:57:05, time: 0.581, data_time: 0.002, memory: 9942, heatmap_loss: 0.0016, acc_pose: 0.0183, loss: 0.0016
2022-10-15 20:21:32,907 - mmpose - INFO - Epoch [1][20/166]	lr: 1.948e-05, eta: 4:14:42, time: 0.579, data_time: 0.002, memory: 9942, heatmap_loss: 0.0016, acc_pose: 0.0236, loss: 0.0016
2022-10-15 20:21:35,772 - mmpose - INFO - Epoch [1][25/166]	lr: 2.448e-05, eta: 3:49:00, time: 0.573, data_time: 0.002, memory: 9942, heatmap_loss: 0.0016, acc_pose: 0.0401, loss: 0.0016
2022-10-15 20:21:38,745 - mmpose - INFO - Epoch [1][30/166]	lr: 2

...

2022-10-15 22:05:49,557 - mmpose - INFO - Epoch [61][5/166]	lr: 5.000e-05, eta: 0:33:29, time: 1.358, data_time: 0.775, memory: 9942, heatmap_loss: 0.0005, acc_pose: 0.8618, loss: 0.0005
2022-10-15 22:05:52,471 - mmpose - INFO - Epoch [61][10/166]	lr: 5.000e-05, eta: 0:33:26, time: 0.583, data_time: 0.003, memory: 9942, heatmap_loss: 0.0005, acc_pose: 0.8684, loss: 0.0005
2022-10-15 22:05:55,364 - mmpose - INFO - Epoch [61][15/166]	lr: 5.000e-05, eta: 0:33:23, time: 0.579, data_time: 0.003, memory: 9942, heatmap_loss: 0.0005, acc_pose: 0.8630, loss: 0.0005
2022-10-15 22:05:58,264 - mmpose - INFO - Epoch [61][20/166]	lr: 5.000e-05, eta: 0:33:20, time: 0.580, data_time: 0.002, memory: 9942, heatmap_loss: 0.0005, acc_pose: 0.8702, loss: 0.0005
2022-10-15 22:06:01,154 - mmpose - INFO - Epoch [61][25/166]	lr: 5.000e-05, eta: 0:33:17, time: 0.578, data_time: 0.002, memory: 9942, heatmap_loss: 0.0005, acc_pose: 0.8826, loss: 0.0005
2022-10-15 22:06:04,046 - mmpose - INFO - Epoch [61][30/166]	

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 757/757, 67.4 task/s, elapsed: 11s, ETA:     0sLoading and preparing results...
DONE (t=0.03s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *keypoints*
DONE (t=0.19s).
Accumulating evaluation results...


2022-10-15 22:14:14,659 - mmpose - INFO - The previous best checkpoint /notebooks/work_dirs/res152_acino_256x256/best_AP_epoch_55.pth was removed


DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] =  0.683
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] =  0.979
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] =  0.805
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] =  0.555
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] =  0.697
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] =  0.729
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] =  0.987
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] =  0.840
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] =  0.633
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] =  0.742


2022-10-15 22:14:18,519 - mmpose - INFO - Now best checkpoint is saved as best_AP_epoch_65.pth.
2022-10-15 22:14:18,522 - mmpose - INFO - Best AP is 0.6828 at 65 epoch.
2022-10-15 22:14:18,578 - mmpose - INFO - Epoch(val) [65][24]	AP: 0.6828, AP .5: 0.9791, AP .75: 0.8055, AP (M): 0.5551, AP (L): 0.6969, AR: 0.7285, AR .5: 0.9868, AR .75: 0.8402, AR (M): 0.6330, AR (L): 0.7416
2022-10-15 22:14:25,633 - mmpose - INFO - Epoch [66][5/166]	lr: 5.000e-05, eta: 0:25:05, time: 1.343, data_time: 0.764, memory: 9942, heatmap_loss: 0.0005, acc_pose: 0.8801, loss: 0.0005
2022-10-15 22:14:28,524 - mmpose - INFO - Epoch [66][10/166]	lr: 5.000e-05, eta: 0:25:02, time: 0.578, data_time: 0.002, memory: 9942, heatmap_loss: 0.0004, acc_pose: 0.8936, loss: 0.0004
2022-10-15 22:14:31,413 - mmpose - INFO - Epoch [66][15/166]	lr: 5.000e-05, eta: 0:24:59, time: 0.578, data_time: 0.002, memory: 9942, heatmap_loss: 0.0004, acc_pose: 0.8812, loss: 0.0004
2022-10-15 22:14:34,302 - mmpose - INFO - Epoch [66][20/1

##### Train model from cfg: 

In [3]:
# Train from config file
def train_from_config(cfg,pretr=True):
    if pretr==False:
        cfg.model['pretrained']=None
    elif pretr==True:
        cfg.work_dir=cfg.work_dir+'_pretr'
        
    mmcv.mkdir_or_exist(cfg.work_dir)
    cfg.log_name = time.strftime('%d-%m-', time.localtime())+"ep"+str(cfg.total_epochs)
    cfg.log_file = os.path.join(cfg.work_dir, f'{cfg.log_name}.log')
    
    autoscale_lr = False # automatically scale lr with the number of gpus
    launcher = 'none' # Job launcher. ['none', 'pytorch', 'slurm', 'mpi']
    deterministic = False # Whether to set deterministic options for CUDNN backend.
    diff_seed = False # Whether or not set different seeds for different ranks
    seed = 0 

    if autoscale_lr:
        # apply the linear scaling rule (https://arxiv.org/abs/1706.02677)
        cfg.optimizer['lr'] = cfg.optimizer['lr'] * len(cfg.gpu_ids) / 8

    if launcher == 'none':
        distributed = False
        if len(cfg.gpu_ids) > 1:
            warnings.warn(
                f'We treat {cfg.gpu_ids} as gpu-ids, and reset to '
                f'{cfg.gpu_ids[0:1]} as gpu-ids to avoid potential error in '
                'non-distribute training time.')
            cfg.gpu_ids = cfg.gpu_ids[0:1]
    else:
        distributed = True
        init_dist(launcher, **cfg.dist_params)
        # re-set gpu_ids with distributed training mode
        _, world_size = get_dist_info()
        cfg.gpu_ids = range(world_size)

    # set multi-process settings
    setup_multi_processes(cfg)

    # init the logger before other steps
    logger = get_root_logger(log_file=cfg.log_file, log_level=cfg.log_level)

    # init the meta dict to record some important information such as
    # environment info and seed, which will be logged
    meta = dict()

    # log env info
    env_info_dict = collect_env()
    env_info = '\n'.join([(f'{k}: {v}') for k, v in env_info_dict.items()])
    dash_line = '-' * 60 + '\n'
    # logger.info('Environment info:\n' + dash_line + env_info + '\n' +
    #             dash_line)
    meta['env_info'] = env_info

    # log some basic info
    logger.info(f'Distributed training: {distributed}')
    # logger.info(f'Config:\n{cfg.pretty_text}')

    # set random seeds
    seed = init_random_seed(seed)
    seed = seed + dist.get_rank() if diff_seed else seed
    logger.info(f'Set random seed to {seed}, '
                f'deterministic: {deterministic}')
    set_random_seed(seed, deterministic=deterministic)
    cfg.seed = seed
    meta['seed'] = seed

    # build dataset
    train_set = [build_dataset(cfg.data.train)]

    # build model
    model = build_posenet(cfg.model)

    if len(cfg.workflow) == 2:
        val_dataset = copy.deepcopy(cfg.data.val)
        val_dataset.pipeline = cfg.data.train.pipeline
        train_set.append(build_dataset(val_dataset))

    if cfg.checkpoint_config is not None:
        # save mmpose version, config file content
        # checkpoints as meta data
        cfg.checkpoint_config.meta = dict(
            mmpose_version=__version__ + get_git_hash(digits=7),
            # config=cfg.pretty_text,
        )

    timestamp = time.strftime('%d-%m-', time.localtime())+"ep"+str(cfg.total_epochs)

    # train model
    train_model(
        model,
        train_set, 
        cfg, 
        distributed=distributed, 
        validate=True, 
        timestamp=timestamp,
        meta=meta)

##### Test 2D pose estimator:

In [ ]:
launcher='none' #job launcher ['none', 'pytorch', 'slurm', 'mpi']
checkpoint=cfg.work_dir+'/latest.pth'
evaluate='mAP' #evaluation metric, which depends on the dataset, "mAP"
fuse_convbn=True
gpu_id=cfg.gpu_ids[0] #only applicable to non-distributed testing
gpu_collect=True #whether to use gpu to collect results
local_rank=0
tmpdir=None
    
# set multi-process settings
setup_multi_processes(cfg)

# set cudnn_benchmark
if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True
cfg.model.pretrained = None ####
cfg.data.test.test_mode = True ####

# init distributed env first, since logger depends on the dist info.
if launcher == 'none':
    distributed = False
else:
    distributed = True
    init_dist(launcher, **cfg.dist_params)

# build the dataloader
test_set = build_dataset(cfg.data.test, dict(test_mode=True))

# step 1: give default values and override (if exist) from cfg.data
loader_cfg = {
    **dict(seed=cfg.get('seed'), drop_last=False, dist=distributed),
    **({} if torch.__version__ != 'parrots' else dict(
           prefetch_num=2,
           pin_memory=False,
       )),
    **dict((k, cfg.data[k]) for k in [
               'seed',
               'prefetch_num',
               'pin_memory',
               'persistent_workers',
           ] if k in cfg.data)
}
# step2: cfg.data.test_dataloader has higher priority
test_loader_cfg = {
    **loader_cfg,
    **dict(shuffle=False, drop_last=False),
    **dict(workers_per_gpu=cfg.data.get('workers_per_gpu', 1)),
    **dict(samples_per_gpu=cfg.data.get('samples_per_gpu', 1)),
    **cfg.data.get('test_dataloader', {})
}

data_loader = build_dataloader(test_set, **test_loader_cfg)



# build the model and load checkpoint
model = build_posenet(cfg.model)
fp16_cfg = cfg.get('fp16', None)
if fp16_cfg is not None:
    wrap_fp16_model(model)

load_checkpoint(model, checkpoint, map_location='cpu')   
if fuse_convbn:
    model = fuse_conv_bn(model)
if not distributed:
    model = MMDataParallel(model, device_ids=[gpu_id])
    outputs = single_gpu_test(model, data_loader)
else:
    model = MMDistributedDataParallel(
        model.cuda(),
        device_ids=[torch.cuda.current_device()],
        broadcast_buffers=False)
    outputs = multi_gpu_test(model, data_loader, tmpdir,
                             gpu_collect)
    

rank, _ = get_dist_info()
eval_config = cfg.get('evaluation', {})
# eval_config = merge_configs(eval_config, dict(metric=evaluate))

out_file = cfg.work_dir+'/test_results.json'

if rank == 0:
    if out_file:
        print(f'\nwriting results to {out_file}') 
        mmcv.dump(outputs, out_file)

    results = test_set.evaluate(outputs, cfg.work_dir, **eval_config) # mmpose/datasets/datasets/animal/... evaluate()
    for k, v in sorted(results.items()):
        print(f'{k}: {v}')

##### 2D Estimator inference, analysis:

In [ ]:
def test_pose_estimator(ckpt,cfg):

    # set multi-process settings
    setup_multi_processes(cfg)

    # set cudnn_benchmark
    if cfg.get('cudnn_benchmark', False):
        torch.backends.cudnn.benchmark = True
    cfg.model.pretrained = None
    cfg.data.test.test_mode = True

    distributed = False

    # build the dataloader
    dataset = build_dataset(cfg.data.test, dict(test_mode=True))
    # step 1: give default values and override (if exist) from cfg.data
    loader_cfg = {
        **dict(seed=cfg.get('seed'), drop_last=False, dist=distributed),
        **({} if torch.__version__ != 'parrots' else dict(
               prefetch_num=2,
               pin_memory=False,
           )),
        **dict((k, cfg.data[k]) for k in [
                   'seed',
                   'prefetch_num',
                   'pin_memory',
                   'persistent_workers',
               ] if k in cfg.data)
    }
    # step2: cfg.data.test_dataloader has higher priority
    test_loader_cfg = {
        **loader_cfg,
        **dict(shuffle=False, drop_last=False),
        **dict(workers_per_gpu=cfg.data.get('workers_per_gpu', 1)),
        **dict(samples_per_gpu=cfg.data.get('samples_per_gpu', 1)),
        **cfg.data.get('test_dataloader', {})
    }
    
    data_loader = build_dataloader(dataset, **test_loader_cfg)

    # build the model and load checkpoint
    model = build_posenet(cfg.model)
    fp16_cfg = cfg.get('fp16', None)
    if fp16_cfg is not None:
        wrap_fp16_model(model)
    load_checkpoint(model, ckpt, map_location='cpu')
    model = fuse_conv_bn(model)

    if not distributed:
        model = MMDataParallel(model, device_ids=[0])
        outputs = single_gpu_test(model, data_loader)

    rank, _ = get_dist_info()
    eval_config = cfg.get('evaluation', {})

    results = dataset.evaluate(outputs, cfg.work_dir, **eval_config)
    for k, v in sorted(results.items()):
        print(f'{k}: {v}')

In [ ]:
# Test model:

pose_checkpoint = cfg.work_dir+'/latest.pth'
json_file_gt='data/acino/annotations/acino_all.json'

# with open('./work_dirs/hrnet_w32_acino-10k_256x256_epoch50/result_keypoints.json','r') as f: 
#     json_file_pred=json.load(f)
# f.close()

# print()

coco = COCO(json_file_gt)
# print(coco)
img_keys = list(coco.imgs.keys())
# print(img_keys)
# # initialize pose model
pose_model = init_pose_model(cfg, pose_checkpoint)
dataset = pose_model.cfg.data.test.type
dataset_info = pose_model.cfg.data.test.get('dataset_info', None)
dataset_info = DatasetInfo(dataset_info)
return_heatmap=True
output_layer_names=None

n=1 # Num imgs to plot
ntest=len(img_keys)
show= np.random.choice(ntest,n)

#GT:
for i in range(n):
    img_id=img_keys[show[i]]
    image=coco.loadImgs(img_id)[0]
    img_name=os.path.join('data/acino/data',image['file_name'])
    
    # pred=next(inst for inst in json_file_pred if inst['image_id']==img_id)
    
    
    ann_ids=coco.getAnnIds(img_id)
    # print(ann_ids)
    ann=coco.anns[ann_ids[0]]
    # print(ann)
    animal_bbox=[]
    gt_keypoints=[]
    
    for ann_id in ann_ids:
        animal={}  
        ann=coco.anns[ann_id]
        animal['bbox']=ann['bbox']
        animal_bbox.append(animal)
        
        gt={}
        x=ann['bbox'][0]
        y=ann['bbox'][1]
        w=ann['bbox'][2]
        h=ann['bbox'][3]
        gt['bbox']=[x,y,x+w,y+h]
        gt['keypoints']=np.array(ann['keypoints']).reshape(24,3)
        gt_keypoints.append(gt)
    
    # x=ann['bbox'][0]
    # y=ann['bbox'][1]
    # w=ann['bbox'][2]
    # h=ann['bbox'][3]
#     print(ann['bbox'])
    # bbox=[x,y,x+w,y+h]
    
#     print(ann['bbox'])

    pose_results, returned_outputs = inference_top_down_pose_model(
        pose_model,
        img_name,
        person_results=animal_bbox, # BBoxs required for top down
        bbox_thr=None,
        format='xywh',
        dataset=dataset,
        dataset_info=dataset_info,
        return_heatmap=True, #for now
        outputs=output_layer_names)
    
    out_file_pred = os.path.join('vis_result', f'vis_as{img_id}.jpg')
    out_file_gt = os.path.join('vis_result', f'vis_gt_as{img_id}.jpg')
    vis_result_pred=vis_pose_result(pose_model,img_name,pose_results,radius=3,thickness=1,out_file=out_file_pred)
    # vis_result_pred = cv2.resize(vis_result_pred, dsize=None, fx=1.5, fy=1.5)
    
    vis_result_gt=vis_pose_result(pose_model,img_name,gt_keypoints,radius=3,thickness=1,out_file=out_file_gt)
    # vis_result_gt = cv2.resize(vis_result_gt, dsize=None, fx=1.5, fy=1.5)
    # vis_result_gt = vis_result_gt

    plt.figure(figsize=(15,10))
    plt.imshow(vis_result_gt[y-25:y+h+25,x-25:x+w+25])
    plt.show()
    
    plt.figure(figsize=(15,10))
    plt.imshow(vis_result_pred[y-25:y+h+50,x-25:x+w+50])
    plt.show()
    
#     os.makedirs('vis_result', exist_ok=True)
#     out_file = os.path.join('vis_result', f'vis_as{image_id}.jpg')
#     vis_result=vis_pose_result(pose_model,image_name,pose_results,radius=3,thickness=1,out_file=out_file)
#     vis_result = cv2.resize(vis_result, dsize=None, fx=1.5, fy=1.5)
#     # plt.figure(figsize=(15,10))
#     # plt.imshow(vis_result)
#     # plt.show()
    


In [ ]:
!python tools/analysis/analyze_logs.py \
        plot_curve work_dirs/hrnet_w32_acino_256x256_epoch50/None.log.json \
        --keys acc_pose \
        --legend Accuracy \
        --out acino_accuracy.jpg

##### 2D Estimator demo with MMDet

In [ ]:
det_config='MMDet/configs/faster_rcnn_r50_fpn_2x_acino.py'
det_checkpoint='MMDet/epoch_4.pth'

pose_config='configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/acino/res50_acino_256x256.py'
pose_checkpoint='work_dirs/res50_acino_256x256_epoch150/latest.pth'

vid='menyacam3.mp4'#'../storage/cam3.mp4'
out_vid='work_dirs/res50_acino_256x256_epoch150/'
use_multi_frames=False
online=True
thickness=1
radius=4
smooth=True
smooth_filter_cfg='configs/_base_/filters/one_euro.py'
device='cuda:0'
bbox_thr=0.7
kpt_thr=0.5
use_oks_tracking=True
tracking_thr=0.3
euro=0
det_cat_id=1
show=False


print('Initializing model...')
det_model = init_detector(
    det_config, det_checkpoint, device=device.lower())
# build the pose model from a config file and a checkpoint file
pose_model = init_pose_model(
    pose_config, pose_checkpoint, device=device.lower())

dataset = pose_model.cfg.data['test']['type']
dataset_info = pose_model.cfg.data['test'].get('dataset_info', None)
if dataset_info is None:
    warnings.warn(
        'Please set `dataset_info` in the config.'
        'Check https://github.com/open-mmlab/mmpose/pull/663 for details.',
        DeprecationWarning)
else:
    dataset_info = DatasetInfo(dataset_info)
print(dataset_info.skeleton)
# read video
video = mmcv.VideoReader(vid)
assert video.opened, f'Failed to load video file {vid}'

if out_vid == '':
    save_out_video = False
else:
    os.makedirs(out_vid, exist_ok=True)
    save_out_video = True

if save_out_video:
    fps = video.fps
    size = (video.width, video.height)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    videoWriter = cv2.VideoWriter(
        os.path.join(out_vid,
                     f'vis_{os.path.basename(vid)}'), fourcc,
        fps, size)

# frame index offsets for inference, used in multi-frame inference setting
if use_multi_frames:
    assert 'frame_indices_test' in pose_model.cfg.data.test.data_cfg
    indices = pose_model.cfg.data.test.data_cfg['frame_indices_test']

# build pose smoother for temporal refinement
if euro:
    warnings.warn(
        'Argument --euro will be deprecated in the future. '
        'Please use --smooth to enable temporal smoothing, and '
        '--smooth-filter-cfg to set the filter config.',
        DeprecationWarning)
    smoother = Smoother(
        filter_cfg='configs/_base_/filters/one_euro.py', keypoint_dim=2)
elif smooth:
    smoother = Smoother(filter_cfg=smooth_filter_cfg, keypoint_dim=2)
else:
    smoother = None

# whether to return heatmap, optional
return_heatmap = False

# return the output of some desired layers,
# e.g. use ('backbone', ) to return backbone feature
output_layer_names = None

next_id = 0
pose_results = []
print('Running inference...')
for frame_id, cur_frame in enumerate(mmcv.track_iter_progress(video)):
    pose_results_last = pose_results

    # get the detection results of current frame
    # the resulting box is (x1, y1, x2, y2)
    mmdet_results = inference_detector(det_model, cur_frame)

    # keep the person class bounding boxes.
    person_results = process_mmdet_results(mmdet_results, det_cat_id)

    if use_multi_frames:
        frames = collect_multi_frames(video, frame_id, indices,
                                      online)

    # test a single image, with a list of bboxes.
    pose_results, _ = inference_top_down_pose_model(
        pose_model,
        frames if use_multi_frames else cur_frame,
        person_results,
        bbox_thr=bbox_thr,
        format='xyxy',
        dataset=dataset,
        dataset_info=dataset_info,
        return_heatmap=return_heatmap,
        outputs=output_layer_names)

    # get track id for each person instance
    pose_results, next_id = get_track_id(
        pose_results,
        pose_results_last,
        next_id,
        use_oks=use_oks_tracking,
        tracking_thr=tracking_thr,
        sigmas=dataset_info.sigmas)

    # post-process the pose results with smoother
    if smoother:
        pose_results = smoother.smooth(pose_results)

    # show the results
    vis_frame = vis_pose_tracking_result(
        pose_model,
        cur_frame,
        pose_results,
        radius=radius,
        thickness=thickness,
        dataset=dataset,
        dataset_info=dataset_info,
        kpt_score_thr=kpt_thr,
        show=False)

    if show:
        cv2.imshow('Frame', vis_frame)

    if save_out_video:
        videoWriter.write(vis_frame)

    if show and cv2.waitKey(1) & 0xFF == ord('q'):
        break

if save_out_video:
    videoWriter.release()

### 3D

#### Extract all AcinoSet 3D data

In [60]:
def add_cameras(n,calib_file,cameras_dict):
    calib_dict=mmcv.load(calib_file)
    
    for i in range(n):
        R=np.array(calib_dict['cameras'][i]['r'])
        T=np.array(calib_dict['cameras'][i]['t'])
        K=np.array(calib_dict['cameras'][i]['k'][:2])
        k=np.array(calib_dict['cameras'][i]['d'][:2])
        k=np.append(k,[k[1]/2],axis=0)
        p=np.array(calib_dict['cameras'][i]['d'][-2:])
        w=calib_dict['camera_resolution'][0]
        h=calib_dict['camera_resolution'][1]
        
        date=calib_file.split('/')[2]
        
        if 'top' in calib_file.split('/'):
            name=f'{date}_1_cam{i+1}'
            idx=f'{date}_1{i+1}'.replace('_','')
        elif 'bottom' in calib_file.split('/'):
            name=f'{date}_2_cam{i+1}'
            idx=f'{date}_2{i+1}'.replace('_','')
        else:
            assert calib_file.split('/')[2].split('_')[0]=='2019'
            name=f'{date}_cam{i+1}'
            idx=f'{date}_0{i+1}'.replace('_','')
            
        cameras_dict[idx]=dict(R=R,T=T,K=K,k=k,p=p,w=w,h=h,name=name)
        cameras_dict[idx]['id']=idx
        
    return cameras_dict

def get_cam(camera_id,cameras):
    for item in list(cameras.keys()):
        copy=item.replace('_','')
        copy=copy.replace('cam','0')
        if copy==camera_id:
            
            return cameras[item]
        else:
            raise Exception("Check camera dict")

def _get_pose_stats(kps):
    """Get statistic information `mean` and `std` of pose data.
    Args:
        kps (ndarray): keypoints in shape [..., K, C] where K and C is
            the keypoint category number and dimension.
    Returns:
        mean (ndarray): [K, C]
    """
    assert kps.ndim > 2
    K, C = kps.shape[-2:]
    kps = kps.reshape(-1, K, C)
    mean = kps.mean(axis=0)
    std = kps.std(axis=0)
    return mean, std

def get_centers_scales(bboxes,scale_factor=1.2):
    centers = np.stack([(bboxes[:, 0] + bboxes[:, 2]) / 2,(bboxes[:, 1] + bboxes[:, 3]) / 2], axis=1)
    scales = scale_factor * np.max(bboxes[:, 2:] - bboxes[:, :2], axis=1) / 200
    return centers, scales

def get_fte_anns(date,cheetah,action,cam):
    #2D Anns
    anns2d=pd.read_csv(f'data/acino_3d/{date}/{cheetah}/{action}/fte_pw/'+cam.split('.')[0]+'_fte.csv')
    anns2d.columns=anns2d.columns+anns2d.iloc[0] # Make keypoint labels column headers
    anns2d=anns2d.iloc[1:,1:]
    kp2d=[]
    
    for img in range(len(anns2d)):
        for kp in range(0,len(anns2d.columns),3):
            if math.isnan(float(anns2d.iloc[img,kp])): 
                kp2d.append(0)
                kp2d.append(0)
                kp2d.append(bool(0))
            else:
                kp2d.append(float(anns2d.iloc[img,kp]))           
                kp2d.append(float(anns2d.iloc[img,kp+1]))
                kp2d.append(bool(1))
                
    kp2d=np.array(kp2d,dtype=float).reshape(len(anns2d),20,3)
    bboxes = np.stack([np.min(kp2d[:, :, 0], axis=1),np.min(kp2d[:, :, 1], axis=1),np.max(kp2d[:, :, 0], axis=1),np.max(kp2d[:, :, 1], axis=1)],axis=1)
    return kp2d,bboxes

def generate_stats(kp2darr,kp3darr,dirr):
    
    kp3darr = kp3darr[..., :3]  # remove visibility
    mean_3d, std_3d = _get_pose_stats(kp3darr)

    kp2darr = kp2darr[..., :2]  # remove visibility
    mean_2d, std_2d = _get_pose_stats(kp2darr)

    
    # centered around root
    # the root keypoint is 0-index
    kps_3d_rel = kp3darr[..., 1:, :] - kp3darr[..., :1, :]
    mean_3d_rel, std_3d_rel = _get_pose_stats(kps_3d_rel)

    kps_2d_rel = kp2darr[..., 1:, :] - kp2darr[..., :1, :]
    mean_2d_rel, std_2d_rel = _get_pose_stats(kps_2d_rel)

    stats = {
        'joint3d_stats': {
            'mean': mean_3d,
            'std': std_3d
        },
        'joint2d_stats': {
            'mean': mean_2d,
            'std': std_2d
        },
        'joint3d_rel_stats': {
            'mean': mean_3d_rel,
            'std': std_3d_rel
        },
        'joint2d_rel_stats': {
            'mean': mean_2d_rel,
            'std': std_2d_rel
        }
    }

    for name, stat_dict in stats.items():
        out_file = f'data/acino_3d/annotations/{dirr}/{name}.pkl'


        with open(out_file, 'wb') as f:
            pickle.dump(stat_dict, f)
        print(f'Create statistic data file: {out_file}')

def get_anns(cam_id, cheetah, action, cam, date, start_frame, end_frame, pose_model,det_model):
    ''' Calculate centers, scales and estimate 2D keypoints using top-down pose-estimator (pose-detect + '''
    
    #DLC 2D Anns:
    kp2d_fte, bboxes_fte = get_fte_anns(date,cheetah,action,cam)
    
    #Save frames
    imgnames=[]
    bboxes_res=np.array([])
    
    kp2d_res=np.empty((0,3), float)
    
    video=mmcv.VideoReader(f'data/acino_3d/{date}/{cheetah}/{action}/{cam}')
    
    next_id=0
    pose_results=[]
        
    for frame_id, cur_frame in enumerate(mmcv.track_iter_progress(video)):
        if frame_id>=start_frame and frame_id<end_frame: 
            
            pose_results_last=pose_results
            
            imgname=f'{cheetah}'+date.split('_')[1]+(date.split('_')[2])+f'_{action}.{cam_id}_{frame_id}.jpg'
            mmcv.imwrite(video[frame_id],f'data/acino_3d/images/{imgname}')  # save frame as JPEG file      
            imgnames.append(imgname)
            
            det_results=inference_detector(det_model,cur_frame)
            cheetah_results=process_mmdet_results(det_results,1)
            
            pose_results,_=inference_top_down_pose_model(
                pose_model,
                cur_frame,
                cheetah_results,
                # bbox_thr=0.3,
                format='xyxy',
                dataset=pose_dataset,
                dataset_info=pose_dataset_info,
                return_heatmap=False,
                outputs=None)
            
            if frame_id==100:
                print("\n",pose_results,"\n")
            
            pose_results, _ = get_track_id(
                    pose_results,
                    pose_results_last,
                    next_id,
                    use_oks=True,
                    tracking_thr=0.7,
                    sigmas=pose_dataset_info.sigmas)
            
            if not pose_results==[]:                
                bboxes_res=np.append(bboxes_res,pose_results[0]['bbox'][:4])
                keypoints=pose_results[0]['keypoints']
                keypoints_new=keypoints[[2,1,0,3,
                                    23,4,18,17,
                                    8,9,19,
                                    5,6,20,
                                    14,15,21,
                                    11,12,22]]
                kp2d_res=np.append(kp2d_res,keypoints_new,axis=0)
                
            else:
                bboxes_res=np.append(bboxes_res,bboxes_fte[frame_id-start_frame])
                kp2d_res=np.append(kp2d_res,kp2d_fte[frame_id-start_frame],axis=0)
            
    
    imgnames=np.array(imgnames)        
    
    #2D Anns
    kp2d_res=kp2d_res.reshape(len(kp2d_res)//20 ,20,3)        
    bboxes_res=bboxes_res.reshape(len(bboxes_res)//4,4)
    
    scale_factor=1.2
    centers_res,scales_res= get_centers_scales(bboxes_res,scale_factor)
    centers_fte,scales_fte= get_centers_scales(bboxes_fte,scale_factor)

    #3D Anns
    anns3d=mmcv.load(f'data/acino_3d/{date}/{cheetah}/{action}/fte_pw/fte.pickle')
    kp3d=anns3d['positions']
    kp3d = np.array(kp3d,dtype=float).reshape(len(anns3d['positions']),20,3)
    kp3d = np.concatenate([kp3d, np.ones((len(kp3d), 20, 1))],
                                axis=2)
    
    assert len(imgnames)==kp3d.shape[0]
    assert kp3d.shape[0]==kp2d_res.shape[0]
    assert kp2d_res.shape==kp2d_fte.shape
    
    return imgnames,centers_res,scales_res,kp2d_res,kp3d,centers_fte,scales_fte,kp2d_fte

In [59]:
det_model=init_detector('MMDet2/configs/faster_rcnn_r50_fpn_2x_acino.py',
                        'work_dirs/acino_rcnn_r50_fpn_2x_ep5/epoch_5.pth')

pose_model=init_pose_model('configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/acino/res50_acino_256x256.py',
                       'work_dirs/res50_acino_256x256_epoch150/epoch_150.pth')
pose_dataset=pose_model.cfg.data['test']['type']
pose_dataset_info = pose_model.cfg.data['test'].get('dataset_info', None)
pose_dataset_info=DatasetInfo(pose_dataset_info)
imgnames,centers_res,scales_res,kp2d_res,kp3d,centers_fte,scales_fte,kp2d_fte=get_anns('2019030502','jules','run','cam2.mp4','2019_03_05',57,176,pose_model,det_model)

load checkpoint from local path: work_dirs/acino_rcnn_r50_fpn_2x_ep5/epoch_5.pth
load checkpoint from local path: work_dirs/res50_acino_256x256_epoch150/epoch_150.pth
[>>>>>>                         ] 58/261, 62.3 task/s, elapsed: 1s, ETA:     3s

/usr/local/lib/python3.9/dist-packages/mmdet/datasets/utils.py:66: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(


[>>>>>>>>>>>                   ] 100/261, 4.6 task/s, elapsed: 22s, ETA:    35s
 [{'bbox': array([583.51965 , 646.1556  , 966.57806 , 833.12256 ,   0.997186],
      dtype=float32), 'keypoints': array([[9.1906982e+02, 6.7043414e+02, 8.6101341e-01],
       [9.0784741e+02, 6.7043414e+02, 9.1241777e-01],
       [9.1906982e+02, 6.8539734e+02, 8.7841564e-01],
       [8.6295776e+02, 6.5547095e+02, 8.5051835e-01],
       [6.9462152e+02, 6.8165656e+02, 9.2805386e-01],
       [8.5921692e+02, 7.0410138e+02, 2.4618514e-01],
       [8.4425378e+02, 7.3028705e+02, 3.9291036e-01],
       [8.8166174e+02, 7.6021350e+02, 9.7117722e-01],
       [8.5547620e+02, 6.9661975e+02, 8.7167418e-01],
       [8.4799451e+02, 7.3028705e+02, 9.1639972e-01],
       [9.2655139e+02, 7.5273187e+02, 9.4218391e-01],
       [6.8339911e+02, 7.2654620e+02, 8.4828985e-01],
       [6.8339911e+02, 7.4899103e+02, 8.8978755e-01],
       [6.3102783e+02, 7.9013989e+02, 8.6944032e-01],
       [7.1332556e+02, 7.1158301e+02, 8.0474102e-0

In [5]:
#Cameras
cameras={}
for date in os.listdir('data/acino_3d'):
    if date.split('_')[0]=='2019':
        #Cameras
        if osp.exists(f'data/acino_3d/{date}/extrinsic_calib/6_cam_scene.json'):
            calib=f'data/acino_3d/{date}/extrinsic_calib/6_cam_scene.json'
            n=6 # cam scene
            add_cameras(n,calib,cameras)

out_file='data/acino_3d/annotations/cameras.pkl'
with open(out_file, 'wb') as fout:
    pickle.dump(cameras, fout)
print(f'Camera parameters have been written to "{out_file}".\n')

Camera parameters have been written to "data/acino_3d/annotations/cameras.pkl".



In [41]:
det_model=init_detector('MMDet2/configs/faster_rcnn_r50_fpn_2x_acino.py',
                        'work_dirs/acino_rcnn_r50_fpn_2x_ep5/epoch_5.pth')

pose_model=init_pose_model('configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/acino/res50_acino_256x256.py',
                       'work_dirs/res50_acino_256x256_epoch150/epoch_150.pth')
pose_dataset=pose_model.cfg.data['test']['type']
pose_dataset_info = pose_model.cfg.data['test'].get('dataset_info', None)
pose_dataset_info=DatasetInfo(pose_dataset_info)
    
imgnames_all_train=[]
kps3d_all_train=[]
centers_all_train_fte=[]
scales_all_train_fte=[]
kps2d_all_train_fte=[]
centers_all_train_res=[]
scales_all_train_res=[]
kps2d_all_train_res=[]

imgnames_all_val=[]
kps3d_all_val=[]
centers_all_val_fte=[]
scales_all_val_fte=[]
kps2d_all_val_fte=[]
centers_all_val_res=[]
scales_all_val_res=[]
kps2d_all_val_res=[]

imgnames_all_test=[]
kps3d_all_test=[]
centers_all_test_fte=[]
scales_all_test_fte=[]
kps2d_all_test_fte=[]
centers_all_test_res=[]
scales_all_test_res=[]
kps2d_all_test_res=[]

test_count=0
train_count=0
val_count=0

mode='train'

for date in os.listdir('data/acino_3d'):
    
    if date.split('_')[0]=='2019':
        
        #Images
        if not osp.exists(f'data/acino_3d/{date}/processed.txt'):
            for cheetah in os.listdir(f'data/acino_3d/{date}'):
                if cheetah != 'extrinsic_calib':
                    # mmcv.mkdir_or_exist(f'{cheetah}')

                    for action in os.listdir(f'data/acino_3d/{date}/{cheetah}'):
                        if action== '.ipynb_checkpoints':
                            continue
                        reconstr_params=mmcv.load(f'data/acino_3d/{date}/{cheetah}/{action}/fte_pw/reconstruction_params.json')
                        start_frame=reconstr_params['start_frame']
                        end_frame=reconstr_params['end_frame']

                        for cam in os.listdir(f'data/acino_3d/{date}/{cheetah}/{action}'):
                            
                            if cam.endswith('.mp4'):

                                cam_name=cam.split('.')[0]
                                cam_id=date.replace('_','')+'0'+cam_name[-1]
                                
                                imgnames,centers_res,scales_res,kp2d_res,kp3d,centers_fte,scales_fte,kp2d_fte = get_anns(cam_id, 
                                                                                                                               cheetah, 
                                                                                                                               action, cam, 
                                                                                                                               date, start_frame, 
                                                                                                                               end_frame,
                                                                                                                               pose_model,
                                                                                                                               det_model)
                         
                                if mode=='train':
                                    imgnames_all_train.append(imgnames)
                                    kps3d_all_train.append(kp3d)
                                    
                                    centers_all_train_res.append(centers_res)
                                    scales_all_train_res.append(scales_res)
                                    kps2d_all_train_res.append(kp2d_res)
                                    
                                    centers_all_train_fte.append(centers_fte)
                                    scales_all_train_fte.append(scales_fte)
                                    kps2d_all_train_fte.append(kp2d_fte)
                                    
                                    print(f'{date}, {mode}, {cheetah}, {action}, {cam}. Train: {train_count}')
                                    train_count+=1
                                    if train_count%6==0:
                                        mode='val'
                                
                                elif mode=='val':
                                    imgnames_all_val.append(imgnames)
                                    kps3d_all_val.append(kp3d)
                                    
                                    centers_all_val_res.append(centers_res)
                                    scales_all_val_res.append(scales_res)
                                    kps2d_all_val_res.append(kp2d_res)
                                    
                                    centers_all_val_fte.append(centers_fte)
                                    scales_all_val_fte.append(scales_fte)
                                    kps2d_all_val_fte.append(kp2d_fte)
                                    
                                    val_count+=1
                                    print(f'{date}, {mode}, {cheetah}, {action}, {cam}. Val: {val_count}')
                                    mode='test'
                                    continue
                                    
                                elif mode=='test':
                                    imgnames_all_test.append(imgnames)
                                    kps3d_all_test.append(kp3d)
                                    
                                    centers_all_test_res.append(centers_res)
                                    scales_all_test_res.append(scales_res)
                                    kps2d_all_test_res.append(kp2d_res)
                                    
                                    centers_all_test_fte.append(centers_fte)
                                    scales_all_test_fte.append(scales_fte)
                                    kps2d_all_test_fte.append(kp2d_fte)
                                    
                                    test_count+=1
                                    print(f'{date}, {mode}, {cheetah}, {action}, {cam}. Test: {test_count}')
                                    
                                    if test_count%2==0:
                                        mode='train'
                                        continue
                                    
            # with open(f'data/acino_3d/{date}/processed.txt','w') as processed:
            #     processed.write(f'{date} processed')

imgnames_all_train = np.concatenate(imgnames_all_train)
kps3d_all_train = np.concatenate(kps3d_all_train)

centers_all_train_res = np.concatenate(centers_all_train_res)
scales_all_train_res = np.concatenate(scales_all_train_res)
kps2d_all_train_res = np.concatenate(kps2d_all_train_res)

centers_all_train_fte = np.concatenate(centers_all_train_fte)
scales_all_train_fte = np.concatenate(scales_all_train_fte)
kps2d_all_train_fte = np.concatenate(kps2d_all_train_fte)

np.savez(file=f'data/acino_3d/annotations/FTE/acino3d_train.npz',imgname=imgnames_all_train,center=centers_all_train_fte,scale=scales_all_train_fte,part=kps2d_all_train_fte,S=kps3d_all_train)
np.savez(file=f'data/acino_3d/annotations/Res50/acino3d_train.npz',imgname=imgnames_all_train,center=centers_all_train_res,scale=scales_all_train_res,part=kps2d_all_train_res,S=kps3d_all_train)
print(f"Saved: train")

generate_stats(kps2d_all_train_res,kps3d_all_train,'Res50')
generate_stats(kps2d_all_train_fte,kps3d_all_train,'FTE')

imgnames_all_val = np.concatenate(imgnames_all_val)
kps3d_all_val = np.concatenate(kps3d_all_val)

centers_all_val_res = np.concatenate(centers_all_val_res)
scales_all_val_res = np.concatenate(scales_all_val_res)
kps2d_all_val_res = np.concatenate(kps2d_all_val_res)

centers_all_val_fte = np.concatenate(centers_all_val_fte)
scales_all_val_fte = np.concatenate(scales_all_val_fte)
kps2d_all_val_fte = np.concatenate(kps2d_all_val_fte)

np.savez(file=f'data/acino_3d/annotations/FTE/acino3d_val.npz',imgname=imgnames_all_val,center=centers_all_val_fte,scale=scales_all_val_fte,part=kps2d_all_val_fte,S=kps3d_all_val)
np.savez(file=f'data/acino_3d/annotations/Res50/acino3d_val.npz',imgname=imgnames_all_val,center=centers_all_val_res,scale=scales_all_val_res,part=kps2d_all_val_res,S=kps3d_all_val)
print(f"Saved: val")

imgnames_all_test = np.concatenate(imgnames_all_test)
kps3d_all_test = np.concatenate(kps3d_all_test)

centers_all_test_fte = np.concatenate(centers_all_test_fte)
scales_all_test_fte = np.concatenate(scales_all_test_fte)
kps2d_all_test_fte = np.concatenate(kps2d_all_test_fte)

centers_all_test_res = np.concatenate(centers_all_test_res)
scales_all_test_res = np.concatenate(scales_all_test_res)
kps2d_all_test_res = np.concatenate(kps2d_all_test_res)


np.savez(file=f'data/acino_3d/annotations/FTE/acino3d_test.npz',imgname=imgnames_all_test,center=centers_all_test_fte,scale=scales_all_test_fte,part=kps2d_all_test_fte,S=kps3d_all_test)
np.savez(file=f'data/acino_3d/annotations/Res50/acino3d_test.npz',imgname=imgnames_all_test,center=centers_all_test_res,scale=scales_all_test_res,part=kps2d_all_test_res,S=kps3d_all_test)

print(f"Saved test")

load checkpoint from local path: work_dirs/acino_rcnn_r50_fpn_2x_ep5/epoch_5.pth
load checkpoint from local path: work_dirs/res50_acino_256x256_epoch150/epoch_150.pth
[>>>>>>                         ] 58/261, 80.7 task/s, elapsed: 1s, ETA:     3s

/usr/local/lib/python3.9/dist-packages/mmdet/datasets/utils.py:66: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 261/261, 6.5 task/s, elapsed: 40s, ETA:     0s
2019_03_05, train, jules, run, cam6.mp4. Train: 0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 261/261, 6.4 task/s, elapsed: 40s, ETA:     0s
2019_03_05, train, jules, run, cam2.mp4. Train: 1
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 261/261, 6.7 task/s, elapsed: 39s, ETA:     0s
2019_03_05, train, jules, run, cam1.mp4. Train: 2
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 261/261, 6.5 task/s, elapsed: 40s, ETA:     0s
2019_03_05, train, jules, run, cam3.mp4. Train: 3
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 271/271, 4.1 task/s, elapsed: 66s, ETA:     0s
2019_03_05, train, jules, flick, cam6.mp4. Train: 4
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 271/271, 3.8 task/s, elapsed: 71s, ETA:     0s
2019_03_05, train, jules, flick, cam2.mp4. Train: 5
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 271/271, 4.0 task/s, elapsed: 68s, ETA:     0s
2019_03_05, val, jules, flick, cam1.mp4. Val: 1
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 271/271, 3.8 task/s, elapsed: 72s, ETA:     0s
2019_03_

#### Train PoseLifter model:

##### PoseLifter with FTE 2D, 3D data

In [3]:
cfg = Config.fromfile('configs/body/3d_kpt_sview_rgb_vid/video_pose_lift/h36m/poselift_acino_27frames_fullconv_supervised.py')
cfg=setup_config(cfg,600,'Res50',causal=True)
train_cfg(cfg)
test_ckpt(cfg,f'{cfg.work_dir}/latest.pth')
demo_pose_lifter('MMDet2/configs/faster_rcnn_r50_fpn_2x_acino.py',
                 'MMDet2/work_dirs/epoch_5.pth',
                 'configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/acino/res50_acino_256x256.py',
                 'work_dirs/res50_acino_256x256_epoch150/epoch_150.pth',
                 cfg, f'{cfg.work_dir}/latest.pth','vis_results/cetanecam3.mp4')
                 

/notebooks/mmpose/utils/setup_env.py:32: UserWarning: Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
/notebooks/mmpose/utils/setup_env.py:42: UserWarning: Setting MKL_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed.
  warnings.warn(
2022-10-12 07:10:36,840 - mmpose - INFO - Distributed training: False
2022-10-12 07:10:36,847 - mmpose - INFO - Set random seed to 0, deterministic: False
2022-10-12 07:10:39,247 - mmpose - INFO - Start running, host: root@nt96m8dshe, work_dir: /notebooks/work_dirs/acino_poselift_Res50_causal_supervised
2022-10-12 07:10:39,250 - mmpose - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) ExpLrUpdaterHook   

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 1713/1713, 1447.7 task/s, elapsed: 1s, ETA:     0s

2022-10-12 07:11:59,907 - mmpose - INFO - Now best checkpoint is saved as best_MPJPE_epoch_10.pth.
2022-10-12 07:11:59,910 - mmpose - INFO - Best MPJPE is 0.4368 at 10 epoch.
2022-10-12 07:11:59,912 - mmpose - INFO - Epoch(val) [10][14]	MPJPE: 0.4367966949939728, MPJPE_flick: 0.43344995379447937, MPJPE_run: 0.44128185510635376, P-MPJPE: 0.2292, P-MPJPE_flick: 0.43344995379447937, P-MPJPE_run: 0.44128185510635376, 3DPCK: 15.3269
2022-10-12 07:12:03,168 - mmpose - INFO - Epoch [11][20/86]	lr: 9.802e-04, eta: 1:02:01, time: 0.157, data_time: 0.108, memory: 185, reg_loss: 0.5089, mpjpe: 0.5089, p_mpjpe: 0.2639, loss: 0.5089
2022-10-12 07:12:04,101 - mmpose - INFO - Epoch [11][40/86]	lr: 9.802e-04, eta: 1:01:30, time: 0.047, data_time: 0.002, memory: 185, reg_loss: 0.5183, mpjpe: 0.5183, p_mpjpe: 0.2694, loss: 0.5183
2022-10-12 07:12:05,052 - mmpose - INFO - Epoch [11][60/86]	lr: 9.802e-04, eta: 1:01:00, time: 0.047, data_time: 0.002, memory: 185, reg_loss: 0.5012, mpjpe: 0.5012, p_mpjpe: 0

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 1713/1713, 1380.4 task/s, elapsed: 1s, ETA:     0s

2022-10-12 07:13:05,602 - mmpose - INFO - Epoch(val) [20][14]	MPJPE: 0.4384212791919708, MPJPE_flick: 0.4945794939994812, MPJPE_run: 0.3631601333618164, P-MPJPE: 0.2110, P-MPJPE_flick: 0.4945794939994812, P-MPJPE_run: 0.3631601333618164, 3DPCK: 16.1588
2022-10-12 07:13:08,767 - mmpose - INFO - Epoch [21][20/86]	lr: 9.608e-04, eta: 0:59:16, time: 0.157, data_time: 0.109, memory: 185, reg_loss: 0.3350, mpjpe: 0.3350, p_mpjpe: 0.1837, loss: 0.3350
2022-10-12 07:13:09,738 - mmpose - INFO - Epoch [21][40/86]	lr: 9.608e-04, eta: 0:59:02, time: 0.049, data_time: 0.002, memory: 185, reg_loss: 0.3384, mpjpe: 0.3384, p_mpjpe: 0.1888, loss: 0.3384
2022-10-12 07:13:10,694 - mmpose - INFO - Epoch [21][60/86]	lr: 9.608e-04, eta: 0:58:48, time: 0.048, data_time: 0.002, memory: 185, reg_loss: 0.3357, mpjpe: 0.3357, p_mpjpe: 0.1804, loss: 0.3357
2022-10-12 07:13:11,652 - mmpose - INFO - Epoch [21][80/86]	lr: 9.608e-04, eta: 0:58:33, time: 0.048, data_time: 0.002, memory: 185, reg_loss: 0.3177, mpjpe: 0

...

2022-10-12 08:20:02,824 - mmpose - INFO - Epoch(val) [590][14]	MPJPE: 0.3127731680870056, MPJPE_flick: 0.3209291398525238, MPJPE_run: 0.30184271931648254, P-MPJPE: 0.1335, P-MPJPE_flick: 0.3209291398525238, P-MPJPE_run: 0.30184271931648254, 3DPCK: 42.6503
2022-10-12 08:20:06,232 - mmpose - INFO - Epoch [591][20/86]	lr: 3.069e-04, eta: 0:00:58, time: 0.157, data_time: 0.108, memory: 185, reg_loss: 0.0654, mpjpe: 0.0654, p_mpjpe: 0.0464, loss: 0.0654
2022-10-12 08:20:07,186 - mmpose - INFO - Epoch [591][40/86]	lr: 3.069e-04, eta: 0:00:57, time: 0.048, data_time: 0.002, memory: 185, reg_loss: 0.0662, mpjpe: 0.0662, p_mpjpe: 0.0460, loss: 0.0662
2022-10-12 08:20:08,127 - mmpose - INFO - Epoch [591][60/86]	lr: 3.069e-04, eta: 0:00:55, time: 0.047, data_time: 0.002, memory: 185, reg_loss: 0.0644, mpjpe: 0.0644, p_mpjpe: 0.0455, loss: 0.0644
2022-10-12 08:20:09,075 - mmpose - INFO - Epoch [591][80/86]	lr: 3.069e-04, eta: 0:00:54, time: 0.047, data_time: 0.002, memory: 185, reg_loss: 0.0647, m

[>>>>>>>>>>>>>>>>>>>>>>>>>>] 1713/1713, 1402.2 task/s, elapsed: 1s, ETA:     0s

2022-10-12 08:21:08,932 - mmpose - INFO - Epoch(val) [600][14]	MPJPE: 0.3005877137184143, MPJPE_flick: 0.3299414813518524, MPJPE_run: 0.2612488269805908, P-MPJPE: 0.1268, P-MPJPE_flick: 0.3299414813518524, P-MPJPE_run: 0.2612488269805908, 3DPCK: 45.3824


load checkpoint from local path: work_dirs/acino_poselift_Res50_causal_supervised/latest.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 3402/3402, 1483.4 task/s, elapsed: 2s, ETA:     0s
writing results to work_dirs/acino_poselift_Res50_causal_supervised/test_results.json
3DPCK: 58.08935761451721
MPJPE: 0.2424284815788269
MPJPE_flick: 0.20270095765590668
MPJPE_run: 0.29161742329597473
P-MPJPE: 0.10129451218471404
P-MPJPE_flick: 0.20270095765590668
P-MPJPE_run: 0.29161742329597473


NameError: name 'demo_pose_lifter' is not defined

In [17]:
test_ckpt(cfg,f'{cfg.work_dir}/latest.pth')

load checkpoint from local path: work_dirs/acino_poselift_Res50_noncausal_supervised/latest.pth
[>>>>>>>>>>>>>>>>>>>>>>>>>>] 3402/3402, 1440.2 task/s, elapsed: 2s, ETA:     0s
writing results to work_dirs/acino_poselift_Res50_noncausal_supervised/test_results.json
3DPCK: 59.216636419296265
MPJPE: 0.2305927574634552
MPJPE_flick: 0.1898818463087082
MPJPE_run: 0.2809993028640747
P-MPJPE: 0.10259932730092741
P-MPJPE_flick: 0.1898818463087082
P-MPJPE_run: 0.2809993028640747


In [ ]:
train_cfg(cfg)
test_ckpt(cfg,f'{cfg.work_dir}/latest.pth')

##### PoseLifter with top-down estimated 2D data, FTE 3D data

In [2]:
def setup_config(cfg,eps,anns2d,causal):
    
    cfg.total_epochs=eps
    
    cfg.data['train']['ann_file']=f'data/acino_3d/annotations/{anns2d}/acino3d_train.npz'
    cfg.data['val']['ann_file']=f'data/acino_3d/annotations/{anns2d}/acino3d_val.npz'
    cfg.data['test']['ann_file']=f'data/acino_3d/annotations/{anns2d}/acino3d_test.npz'
    
    if causal:
        caus='causal'
    else:
        caus='noncausal'
    
    cfg.model['backbone']['causal']=causal
    cfg.work_dir=f'work_dirs/acino_poselift_{anns2d}_{caus}_supervised'

    cfg.log_name = time.strftime('%d-%m-', time.localtime())+"ep"+str(cfg.total_epochs)
    cfg.log_file = os.path.join(cfg.work_dir, f'{cfg.log_name}.log')
    mmcv.mkdir_or_exist(cfg.work_dir)
    cfg.data_cfg['causal']=causal
    # set random seeds
    seed=0
    seed = init_random_seed(seed)
    
    set_random_seed(seed, deterministic=False)
    cfg.seed = seed
    
    return cfg
    
def train_cfg(cfg):
    # set multi-process settings
    setup_multi_processes(cfg)
    # init the logger before other steps
    logger = get_root_logger(log_file=cfg.log_file, log_level=cfg.log_level)
    
    # init the meta dict to record some important information such as
    # environment info and seed, which will be logged
    meta = dict()

    # log env info
    env_info_dict = collect_env()
    env_info = '\n'.join([(f'{k}: {v}') for k, v in env_info_dict.items()])
    dash_line = '-' * 60 + '\n'
    # logger.info('Environment info:\n' + dash_line + env_info + '\n' +
    #             dash_line)
    meta['env_info'] = env_info

    # log some basic info
    logger.info(f'Distributed training: False')
    # logger.info(f'Config:\n{cfg.pretty_text}')
    logger.info(f'Set random seed to {cfg.seed}, '
                f'deterministic: False')
    meta['seed'] = cfg.seed

    # build dataset
    train_set = [build_dataset(cfg.data.train)]

    # build model
    model = build_posenet(cfg.model)
    timestamp = time.strftime('%d-%m-', time.localtime())+"ep"+str(cfg.total_epochs)
    
    # print(cfg.lr_config)
    train_model(
        model,
        train_set, 
        cfg, 
        distributed=False, 
        validate=True, 
        timestamp=timestamp,
        meta=meta)
    
def test_ckpt(cfg,ckpt):
#     cfg.total_epochs=eps
    
#     cfg.data['train']['ann_file']=f'data/acino_3d/annotations/{anns2d}/acino3d_train.npz'
#     cfg.data['val']['ann_file']=f'data/acino_3d/annotations/{anns2d}/acino3d_val.npz'
#     cfg.data['test']['ann_file']=f'data/acino_3d/annotations/{anns2d}/acino3d_test.npz'
    
#     if causal:
#         caus='causal'
#     else:
#         caus='noncausal'
    
#     cfg.model['backbone']['causal']=causal
#     cfg.work_dir=f'work_dirs/acino_poselift_{anns2d}_{caus}_supervised'
    
    distributed = False
    fuse_convbn=True
    
    # build the dataloader
    dataset = build_dataset(cfg.data.test, dict(test_mode=True))
    # step 1: give default values and override (if exist) from cfg.data
    loader_cfg = {
        **dict(seed=cfg.get('seed'), drop_last=False, dist=distributed),
        **({} if torch.__version__ != 'parrots' else dict(
               prefetch_num=2,
               pin_memory=False,
           )),
        **dict((k, cfg.data[k]) for k in [
                   'seed',
                   'prefetch_num',
                   'pin_memory',
                   'persistent_workers',
               ] if k in cfg.data)
    }
    # step2: cfg.data.test_dataloader has higher priority
    test_loader_cfg = {
        **loader_cfg,
        **dict(shuffle=False, drop_last=False),
        **dict(workers_per_gpu=cfg.data.get('workers_per_gpu', 1)),
        **dict(samples_per_gpu=cfg.data.get('samples_per_gpu', 1)),
        **cfg.data.get('test_dataloader', {})
    }
    data_loader = build_dataloader(dataset, **test_loader_cfg)

    # build the model and load checkpoint
    model = build_posenet(cfg.model)
    fp16_cfg = cfg.get('fp16', None)
    if fp16_cfg is not None:
        wrap_fp16_model(model)
    load_checkpoint(model, ckpt, map_location='cpu')

    if fuse_convbn:
        model = fuse_conv_bn(model)

    if not distributed:
        model = MMDataParallel(model, device_ids=[0])
        outputs = single_gpu_test(model, data_loader)
    # else:
    #     model = MMDistributedDataParallel(
    #         model.cuda(),
    #         device_ids=[torch.cuda.current_device()],
    #         broadcast_buffers=False)
    #     outputs = multi_gpu_test(model, data_loader, args.tmpdir,
    #                              args.gpu_collect)

    rank, _ = get_dist_info()
    eval_config = cfg.get('evaluation', {})
    # eval_config = merge_configs(eval_config, dict(metric='3dpck'))

    if rank == 0:
        # if out:
        
        print(f'\nwriting results to {cfg.work_dir}/test_results.json')
        mmcv.dump(outputs, f'{cfg.work_dir}/test_results.json')

        results = dataset.evaluate(outputs, cfg.work_dir, **eval_config)
        for k, v in sorted(results.items()):
            print(f'{k}: {v}')

In [31]:
def demo_pose_lifter(det_cfg,det_ckpt,pose_cfg,pose_ckpt,poselift_cfg,poselift_ckpt,vid):
    # print(cfg.model)
    # print(cfg.data_cfg)
    # print(cfg.data)
    #Params:
    return_heatmap=False
    output_layer_names = None
    use_multi_frames=False
    rebase_keypoint_height=True
    num_instances=1
    pose_results_list = []
    next_id = 0
    pose_results = []
    save_out_video=True
    
    det_model = init_detector(det_cfg,det_ckpt)
    
    pose_model=init_pose_model(pose_cfg,pose_ckpt)
    pose_dataset = pose_model.cfg.data['test']['type']
    pose_dataset_info = pose_model.cfg.data['test'].get('dataset_info', None)
    pose_dataset_info=DatasetInfo(pose_dataset_info)    
    
    poselift_model=init_pose_model(poselift_cfg,poselift_ckpt)
    poselift_dataset = poselift_model.cfg.data['test']['type']
    poselift_dataset_info = poselift_model.cfg.data['test'].get('dataset_info', None)
    poselift_dataset_info = DatasetInfo(poselift_dataset_info)
    
    video = mmcv.VideoReader(vid)
    
    print("\n2D Detection and Pose Estimation:")
    for frame_id, cur_frame in enumerate(mmcv.track_iter_progress(video)):

        pose_results_last = pose_results

        # get the detection results of current frame
        # the resulting box is (x1, y1, x2, y2)
        det_results = inference_detector(det_model, cur_frame)

        # keep the person class bounding boxes.
        det_results = process_mmdet_results(det_results, 1)
        
        pose_results, _ = inference_top_down_pose_model(
            pose_model,
            cur_frame,
            det_results,
            format='xyxy',
            dataset=pose_dataset,
            dataset_info=pose_dataset_info,
            return_heatmap=return_heatmap,
            outputs=output_layer_names)
        
        print("\n",pose_results,"\n")
        
        pose_results, _ = get_track_id(
            pose_results,
            pose_results_last,
            next_id,
            use_oks=True,
            tracking_thr=0.9,
            sigmas=pose_dataset_info.sigmas)
        
        pose_results_list.append(copy.deepcopy(pose_results))
    
    if save_out_video:
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        fps = video.fps
        writer = None

    # Re-arrange keypoints:
    for pose_results in pose_results_list:
        for res in pose_results:
            keypoints = res['keypoints']
            keypoints_new = np.zeros((20, keypoints.shape[1]), dtype=keypoints.dtype)
            keypoints_new=keypoints[[2,1,0,3,
                                    23,4,18,17,
                                    8,9,19,
                                    5,6,20,
                                    14,15,21,
                                    11,12,22]] 
            res['keypoints']=keypoints_new
    
    # smoother=Smoother('configs/_base_/filters/one_euro.py',keypoint_key='keypoints',keypoint_dim=2)
    data_cfg = poselift_model.cfg.data_cfg
    
    print("\nPose Lifting:")
    for i, pose_results in enumerate(mmcv.track_iter_progress(pose_results_list)): #Pose det results: iterate over each frame
        # extract and pad input pose2d sequence
        pose_results_2d = extract_pose_sequence( # returns 2D kp and BBox for 27 frames. len = 27
            pose_results_list,
            frame_idx=i,
            causal=data_cfg.causal,
            seq_len=data_cfg.seq_len,
            step=data_cfg.seq_frame_interval)

        # pose_results_2d = smoother.smooth(pose_results_2d)

        # 2D-to-3D pose lifting
        poselift_results = inference_pose_lifter_model( #'keypoints': 2D keypoints for 27 frames, 'keypoints_3d': 3D keypoints for target frame
            poselift_model,
            pose_results_2d=pose_results_2d,
            dataset=poselift_dataset,
            dataset_info=poselift_dataset_info,
            with_track_id=True,
            image_size=video.resolution,
            norm_pose_2d=False)
        # print(pose_lift_results) #Norm = true


        poselift_results_vis = []
        for idx, res in enumerate(poselift_results):
            keypoints_3d = res['keypoints_3d']
            
            # exchange y,z-axis, and then reverse the direction of x,z-axis
            # keypoints_3d = keypoints_3d[..., [0, 2, 1]]
            # keypoints_3d[..., 0] = -keypoints_3d[..., 0]
            # keypoints_3d[..., 2] = -keypoints_3d[..., 2]
            # rebase height (z-axis)
            
            if rebase_keypoint_height:
                keypoints_3d[..., 2] -= np.min(
                    keypoints_3d[..., 2], axis=-1, keepdims=True)
            res['keypoints_3d'] = keypoints_3d
            # add title
            det_res = pose_results[idx]
            instance_id = det_res['track_id']
            res['title'] = f'Prediction ({instance_id})'
            
            # only visualize the target frame
            res['keypoints'] = det_res['keypoints']
            res['bbox'] = det_res['bbox']
            res['track_id'] = instance_id
            poselift_results_vis.append(res)

        # print(len(poselift_results_vis))
        # Visualization

        # if len(poselift_results_vis)==1:
# 
        if num_instances < 0:
            num_instances = len(poselift_results_vis)

        img_vis = vis_3d_pose_result(
            poselift_model,
            result=poselift_results_vis,
            img=video[i],
            dataset=poselift_dataset,
            dataset_info=poselift_dataset_info,
            out_file=None,
            num_instances=num_instances,
            show=False)

        if save_out_video:
            if writer is None:
                writer = cv2.VideoWriter(f'{cfg.work_dir}/{os.path.basename(vid).split(".")[0]}_out.mp4', fourcc, fps, (img_vis.shape[1], img_vis.shape[0]))
            writer.write(img_vis)

    if save_out_video:
        writer.release()

In [ ]:
demo_pose_lifter('MMDet2/configs/faster_rcnn_r50_fpn_2x_acino.py',
                 'MMDet2/work_dirs/epoch_5.pth',
                 'configs/animal/2d_kpt_sview_rgb_img/topdown_heatmap/acino/res50_acino_256x256.py',
                 'work_dirs/res50_acino_256x256_epoch150/epoch_150.pth',
                 'configs/body/3d_kpt_sview_rgb_vid/video_pose_lift/h36m/poselift_acino_27frames_fullconv_supervised.py',
                 'work_dirs/acino_poselift_Res50_causal_supervised/latest.pth',
                 'vis_results/cetanecam3.mp4')


In [20]:
val=np.load('data/acino_3d/annotations/Res50/acino3d_val.npz')
test=np.load('data/acino_3d/annotations/Res50/acino3d_test.npz')
trainres=np.load('data/acino_3d/annotations/Res50/acino3d_train.npz')
trainfte=np.load('data/acino_3d/annotations/FTE/acino3d_train.npz')
# len(train['S'])+len(test['S'])+len(val['S'])
print(trainres['part'][1089])
trainfte['part'][1089]

[[2.27857129e+03 7.29896179e+02 9.17670071e-01]
 [2.27857129e+03 7.18204224e+02 9.42829192e-01]
 [2.28831445e+03 7.20152893e+02 9.37604725e-01]
 [2.31364697e+03 7.20152893e+02 8.70545506e-01]
 [2.34482568e+03 7.29896179e+02 8.27093482e-01]
 [2.37990137e+03 7.24050232e+02 9.09817994e-01]
 [2.39938794e+03 7.64971985e+02 4.74326760e-01]
 [2.40523389e+03 7.78612610e+02 3.33682388e-01]
 [2.29610913e+03 7.49382751e+02 2.32720345e-01]
 [2.30195508e+03 7.84458557e+02 2.72322237e-01]
 [2.28636597e+03 7.94201843e+02 8.32268119e-01]
 [2.31559570e+03 7.45485413e+02 7.91275978e-01]
 [2.31364697e+03 7.74715271e+02 9.01403725e-01]
 [2.28052002e+03 7.78612610e+02 8.67152929e-01]
 [2.37405542e+03 7.74715271e+02 5.14082372e-01]
 [2.37210669e+03 7.78612610e+02 8.05275679e-01]
 [2.37990137e+03 7.96150513e+02 7.39946544e-01]
 [2.38185010e+03 7.41588135e+02 8.16057563e-01]
 [2.38769604e+03 7.72766663e+02 7.98791528e-01]
 [2.40328540e+03 7.68869324e+02 7.62276888e-01]]


array([[2.29547337e+03, 7.27362749e+02, 1.00000000e+00],
       [2.28947400e+03, 7.23829074e+02, 1.00000000e+00],
       [2.28079335e+03, 7.27664226e+02, 1.00000000e+00],
       [2.31894582e+03, 7.25073121e+02, 1.00000000e+00],
       [2.35034350e+03, 7.28986643e+02, 1.00000000e+00],
       [2.37796376e+03, 7.32330130e+02, 1.00000000e+00],
       [2.39196836e+03, 7.61750823e+02, 1.00000000e+00],
       [2.40850276e+03, 7.87891066e+02, 1.00000000e+00],
       [2.32819659e+03, 7.47335645e+02, 1.00000000e+00],
       [2.32385268e+03, 7.84950122e+02, 1.00000000e+00],
       [2.29544224e+03, 8.19528479e+02, 1.00000000e+00],
       [2.30578287e+03, 7.37516153e+02, 1.00000000e+00],
       [2.28954614e+03, 7.76692886e+02, 1.00000000e+00],
       [2.26193296e+03, 7.80906562e+02, 1.00000000e+00],
       [2.38239902e+03, 7.39020870e+02, 1.00000000e+00],
       [2.40505776e+03, 7.70862529e+02, 1.00000000e+00],
       [2.42241908e+03, 7.88953321e+02, 1.00000000e+00],
       [2.35888265e+03, 7.43684

In [ ]:
def load_pickle(pickle_file):
    """
    Loads a dictionary from a saved skeleton .pickle file
    """
    with open(pickle_file, 'rb') as handle:
        data = pickle.load(handle)

    return(data)

In [ ]:
# data settings
data_root = 'data/acino_3d'
data_cfg = dict(
    num_joints=20,
    seq_len=27,
    seq_frame_interval=1,
    causal=False,
    temporal_padding=True,
    joint_2d_src='gt',
    need_camera_param=True,
    camera_param_file=f'{data_root}/annotations/cameras.pkl',
)

pose_lift_model=init_pose_model('configs/body/3d_kpt_sview_rgb_vid/video_pose_lift/h36m/videopose3d_acino_27frames_fullconv_supervised.py','work_dirs/videopose3d_acino_27frames_epoch160/latest.pth')
pose_lift_dataset = pose_lift_model.cfg.data['test']['type']
pose_lift_dataset_info = pose_lift_model.cfg.data['test'].get('dataset_info', None)
pose_lift_dataset_info=DatasetInfo(pose_lift_dataset_info)
data=np.load('data/acino_3d/annotations/acino3d_train.npz')

if 'scale' in data:
    _scales = data['scale'].astype(np.float32)
else:
    _scales = np.zeros(num_imgs, dtype=np.float32)

if 'center' in data:
    _centers = data['center'].astype(np.float32)
else:
    _centers = np.zeros((num_imgs, 2), dtype=np.float32)
_imgnames = data['imgname']
num_imgs = len(_imgnames)
num_joints=data_cfg['num_joints']
# get 3D pose
if 'S' in data.keys():
    _joints_3d = data['S'].astype(np.float32)
else:
    _joints_3d = np.zeros((num_imgs, num_joints, 4), dtype=np.float32)

# get 2D pose
if 'part' in data.keys():
    _joints_2d = data['part'].astype(np.float32)
else:
    _joints_2d = np.zeros((num_imgs, num_joints, 3), dtype=np.float32)

data_info = {
    'imgnames': _imgnames,
    'joints_3d': _joints_3d,
    'joints_2d': _joints_2d,
    'scales': _scales,
    'centers': _centers,
}

joint_2d_src = data_cfg.get('joint_2d_src', 'gt')
# if joint_2d_src not in self.SUPPORTED_JOINT_2D_SRC:
#     raise ValueError(
#         f'Unsupported joint_2d_src "{self.joint_2d_src}". '
#         f'Supported options are {self.SUPPORTED_JOINT_2D_SRC}')

joint_2d_det_file = data_cfg.get('joint_2d_det_file', None)

need_camera_param = data_cfg.get('need_camera_param', False)

actions = data_cfg.get('actions', '_all_')
actions = set(actions if isinstance(actions, (list, tuple)) else [actions])
subjects = data_cfg.get('subjects', '_all_')

def _parse_h36m_imgname(imgname):
    subj, rest = osp.basename(imgname).split('_', 1)
    action, rest = rest.split('.', 1)
    camera, rest = rest.split('_', 1)
    return subj,action,camera

video_frames = defaultdict(list)
for idx, imgname in enumerate(data_info['imgnames']):
    subj, action, camera = _parse_h36m_imgname(imgname)

    if '_all_' not in actions and action not in actions:
        continue

    if '_all_' not in subjects and subj not in subjects:
        continue

    video_frames[(subj, action, camera)].append(idx)
    
video_frames